# Alignment hypergraph to variant graph with networkx and masked arrays

Convert alignment hypergraph to variant graph before visualization.

## Initialize

In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext memory_profiler

import warnings
warnings.filterwarnings('ignore')

import gc # garbage collection
import operator # comparison operators lt and gt
from collections import defaultdict, deque
from typing import Set, List
from dataclasses import dataclass
import networkx as nx
import re
import time
import queue

import numpy as np
import numpy.ma as ma

import graphviz
from IPython.display import SVG

import pprint
pp = pprint.PrettyPrinter(indent=2)

## Load data

In [2]:
%run create-blocks.ipynb # data will be in prioritized_blocks
print('aligning', how_many_paragraphs, 'paragraphs') # confirm

1 paragraphs from 6 witnesses
aligning 1 paragraphs


## Create data structures

1. Create witness_node (dataclass) for witness nodes. Properties (not all may be needed):
    1. token string: str
    1. witness: str
    1. offset in witness: int
    1. hyperedge: hyperedge instance (initially null)
    1. first_block: int (used for debugging, defaults to None)
1. Create alignment_hyperedge (dataclass) for … wait for it! … alignment hyperedges! Properties:
    1. witness_nodes: set of witness_node objects
1. Create lists for witness_node and alignment_hyperedge instances
1. Create START and END nodes and associated hyperedges

**TODO:** Can we abolish the alignment_hyperedge objects entirely, keeping all information in the vector space model?

In [3]:
@dataclass(unsafe_hash=True)
class WitnessNode:
    """TODO: Write a docstring some day"""
    token_string: str
    witness: str
    witness_offset: int
    hyperedge: hyperedge = None
    # first_block: int = None # TODO: Remove this?

    def __repr__(self):
        return self.token_string

# Subclass of WitnessNodeEnd so that our constructed end node will be able to identify itself
class WitnessNodeEnd(WitnessNode):
    """TODO: Write a docstring some day"""

In [4]:
class AlignmentHyperedge():
    """TODO: Write a real docstring someday"""
    def __init__(self, kwargs):
        self.__dict__.update(kwargs) # overwrite the real value
        self._sigla = [key for key in kwargs.keys()]
    def __repr__(self):
        return "|".join(
            [":".join(
                    [str(key), str(getattr(self, key))]
                ) for key in self._sigla]
        )
    def __setitem__(self, key, value):
        # Value can be updated without adding another copy of the siglum to _sigla
        self.__dict__[key] = value
        if key not in self._sigla:
            self._sigla.append(key)
    def __getitem__(self, key):
        return self.__dict__[key]
    def __contains__(self, key):
        return key in self.__dict__
    def values(self):
        return [self[key] for key in self._sigla]
    def items(self):
        return [(key, self.__dict__[key]) for key in self._sigla]

In [5]:
# Store all witness nodes in dictionary of lists
witness_node_lists = defaultdict(list) # one per witness (keys will be sigla)
# create, for each witness:
#   START node
#   nodes for real tokens
#   END node
for index, witness_siglum in enumerate(witness_sigla):
    # witness_sigla is a global set when the input data is read
    for witness_token_offset, witness_token in enumerate(witnesses[index]): # list of tokens in a single witness
        witness_node_lists[witness_siglum].append(WitnessNode(witness_token, witness_siglum, witness_token_offset))
    witness_node_lists[witness_siglum].append(WitnessNodeEnd('END', witness_siglum, len(token_array)))

In [6]:
# store all hyperedges in list
hyperedges = []
# create hyperedges for START and END
offsets = [0, -1]
for offset in offsets:
    dictionary = {siglum: witness_node_lists[siglum][offset] for siglum in witness_sigla}
    hyperedge = AlignmentHyperedge(dictionary)
    for witness_node in dictionary.values():
        witness_node.hyperedge = hyperedge
    hyperedges.append(hyperedge)

In [7]:
def create_hyperedges(block):
    """Returns hyperedge data for block, together with offset_start_positions"""
    new_hyperedges = []
    for token_offset_in_block in range(block.token_count):
        # first add the token_offset_in_block to the block start positions
        offset_start_positions = [value + token_offset_in_block for value in block.all_start_positions]
        tokens_to_place = [
            (
                witness_sigla[token_membership_array[token_array_index]],
                token_witness_offset_array[token_array_index]
            )
            for token_array_index in offset_start_positions
        ]
        dictionary = { # data for new hyperedge
            siglum: witness_node_lists[siglum][offset + 1]
            for siglum, offset in tokens_to_place
        }
        # then create an AlignmentHyperedge instance for the new hyperedge
        #   and add to list of alignment hyperedges
        hyperedge = AlignmentHyperedge(dictionary)
        new_hyperedges.append(hyperedge)

    return (new_hyperedges)

In [8]:
def convert_hyperedge_to_masked_array(hyperedge, sigla: list):
    """Converts hyperedge to masked array

    hyperedge: hyperedge to convert
        (list of witness nodes, with witness as key and witness offset as value)
    witnesses: list of sigla in fixed order

    Returns: masked array with values masked for witnesses not in hyperedge
    """
    row = []
    mask = []
    for siglum in sigla:
        if siglum in hyperedge: # enter witness offset and set mask to False
            row.append(hyperedge[siglum].witness_offset)
            mask.append(False)
        else:
            row.append(np.nan)
            mask.append(True)
    return ma.MaskedArray(row, mask=mask)

In [9]:
def convert_alignment_graph_to_numpy_masked_array(existing_ag, witness_sigla: list):
    """Create numpy masked array (matrix) from alignment graph

    Horizontal dimension = number of witnesses (global)
    Vertical dimension = number of hyperedges in alignment hypergraph except START and END

    witnesses : list of sigla (global)

    Returns: two-dimensional masked array, with values not in hyperedges masked
    """
    rows = [] # each row is a masked array, one per hyperedge
    for hyperedge in existing_ag[2:]: # skip START and END, which were added first to hyperedges list
        rows.append(convert_hyperedge_to_masked_array(hyperedge, witness_sigla)) # each hyperedge is a masked array
    return ma.MaskedArray(rows) # outer dimension must be masked for inner mask to be seen

In [10]:
def check_whether_okay_to_place(current_hyperedges, potential_hyperedge, debug=None) -> bool:
    """Return True iff we can add row without creating crossing hyperedge

    current_hyperedge: two-dimensional output of convert_alignment_graph_to_numpy_masked_array()
    potential_hyperedge: convert here to masked array

    If subtracting a potential hyperedge from any existing hyperedge would
    return values that diverge in sign, the potential would cross that existing"""
    current_hyperedges_as_ma = convert_alignment_graph_to_numpy_masked_array(current_hyperedges, witness_sigla)
    potential_hyperedge_as_ma = convert_hyperedge_to_masked_array(potential_hyperedge, witness_sigla)
#     print(f"{current_hyperedges_as_ma=}")
#     print(f"{current_hyperedges_as_ma.shape=}")
#     print(f"{potential_hyperedge_as_ma=}")
#     print(f"{potential_hyperedge_as_ma.shape=}")
    subtractionResult = current_hyperedges_as_ma - potential_hyperedge_as_ma
    signs = np.sign(subtractionResult)
    okayToPlace = (signs.min(axis=1) == signs.max(axis=1)).all()
    if debug:
        print(f"{current_hyperedges_as_ma=}")
        print(f"{potential_hyperedge_as_ma=}")
        print(f"{subtractionResult=}")
        print(f"{signs=}")
        print(f"{signs.min(axis = 1)}")
        print(f"{signs.max(axis = 1)}")
        print(f"{okayToPlace=}")
        print(f"{type(okayToPlace)=}")
    return True if okayToPlace is ma.masked else okayToPlace

In [11]:
def merge_hyperedge(potential_hyperedge, debug=False):
    # check whether a witness_node has got the hyperedge property set
    # a hyperedge may contain zero, some, or all witnesses; we need to handle all these cases
    found = set([witnessnode.hyperedge for siglum, witnessnode in potential_hyperedge.items() if witnessnode.hyperedge])
    if len(found) > 1:
        # found two or more existing hyperedges...
        # remove them from the hyperedges set
        for existing_hyperedge in found:
            if debug==True:
                print("We are trying to remove the hyperedge: "+str(existing_hyperedge))
                print("Existing hyperedge has nodes: " + str(existing_hyperedge))
                print("Potential hyperedge has nodes: " + str(potential_hyperedge))
            hyperedges.remove(existing_hyperedge)
            # NB: Siglum may be in both existing and potential; we assume (!) always
            #   with the same value
            # TODO (maybe): Should we override the keys method in Alignment_Hyperedge instead of checking _sigla?
            for siglum in existing_hyperedge._sigla: # don't get all keys, since that would include _sigla
                if siglum not in potential_hyperedge:
                    potential_hyperedge[siglum] = existing_hyperedge[siglum]
        # replace them by a new hyperedge
        hyperedges.append(potential_hyperedge)
        # update hyperedge property on the witness nodes involved in the hyperedge
        new_hyperedge = potential_hyperedge
        for witness_node in new_hyperedge.values():
            witness_node.hyperedge = new_hyperedge

    elif len(found) == 0:
        # we checked for transpositions
        # TODO: Can we filter out hyperedges we've already placed, so that we don't have to process them
        #   fully each time
        hyperedges.append(potential_hyperedge)
        # update hyperedge property on the witness nodes involved in the hyperedge
        new_hyperedge = potential_hyperedge
        for witness_node in new_hyperedge.values():
            witness_node.hyperedge = new_hyperedge
    else: # update single existing hyperedge
        for siglum in potential_hyperedge._sigla:
            list(found)[0][siglum] = potential_hyperedge[siglum] # add node to hyperedge (possibly redundantly)
            potential_hyperedge[siglum].hyperedge = list(found)[0] # and update hyperedge property of node

In [12]:
# create all hyperedges (gingerly)
# We exclude blocks with repetition (temporarily?)
selected_blocks = filter(lambda x: len(x.all_start_positions) == x.witness_count, prioritized_blocks)

# %prun profiles; see https://towardsdatascience.com/magic-commands-for-profiling-in-jupyter-notebook-d2ef00e29a63
def process_blocks(selected_blocks):
    for index, selected_block in enumerate(selected_blocks):
        # Create hyperedges for entire block
        new_hyperedges = create_hyperedges(selected_block)
        # Check only first and last, merge is both are okay
        # NB: Checking only first and last to test entire block will break with discontinuous blocks
        #   (should we ever switch to working with discontinuous blocks)
        if index == 0:
            # add the first block without checking for transposition since matrix will be empty
            for potential_hyperedge in new_hyperedges:
                merge_hyperedge(potential_hyperedge)
        else:
            debug_flag = None
            no_transposition = check_whether_okay_to_place(hyperedges, new_hyperedges[0], debug_flag) and check_whether_okay_to_place(hyperedges, new_hyperedges[-1])
            if no_transposition: # all-or-nothing
                print("Block #" + str(index) + " has no transposition")
                for potential_hyperedge in new_hyperedges:
                    merge_hyperedge(potential_hyperedge, debug=False)
            else:
                print("Block #"+ str(index) + " (allegedly) has a transposition")

# %prun process_blocks(selected_blocks)
process_blocks(selected_blocks)

Block #1 has no transposition
Block #2 has no transposition
Block #3 has no transposition
Block #4 has no transposition
Block #5 has no transposition
Block #6 has no transposition
Block #7 has no transposition
Block #8 has no transposition
Block #9 has no transposition
Block #10 has no transposition
Block #11 has no transposition
Block #12 has no transposition


Block #13 has no transposition
Block #14 has no transposition
Block #15 has no transposition
Block #16 has no transposition
Block #17 has no transposition
Block #18 has no transposition
Block #19 has no transposition
Block #20 has no transposition


Block #21 has no transposition
Block #22 has no transposition
Block #23 has no transposition
Block #24 has no transposition
Block #25 has no transposition
Block #26 has no transposition
Block #27 has no transposition
Block #28 has no transposition
Block #29 has no transposition
Block #30 has no transposition
Block #31 has no transposition


Block #32 has no transposition
Block #33 has no transposition
Block #34 has no transposition
Block #35 has no transposition
Block #36 has no transposition
Block #37 has no transposition
Block #38 has no transposition
Block #39 has no transposition
Block #40 has no transposition


Block #41 has no transposition
Block #42 has no transposition
Block #43 has no transposition
Block #44 has no transposition
Block #45 has no transposition
Block #46 has no transposition
Block #47 has no transposition
Block #48 has no transposition


...

Block #894 has no transposition
Block #895 has no transposition
Block #896 has no transposition
Block #897 has no transposition
Block #898 has no transposition
Block #899 has no transposition
Block #900 has no transposition


Block #901 has no transposition
Block #902 has no transposition
Block #903 has no transposition
Block #904 has no transposition
Block #905 has no transposition
Block #906 has no transposition
Block #907 has no transposition
Block #908 has no transposition


Block #909 has no transposition
Block #910 has no transposition
Block #911 has no transposition
Block #912 has no transposition
Block #913 has no transposition
Block #914 has no transposition


Block #915 has no transposition
Block #916 has no transposition
Block #917 has no transposition
Block #918 has no transposition
Block #919 has no transposition
Block #920 has no transposition
Block #921 has no transposition


Block #922 has no transposition
Block #923 has no transposition
Block #924 has no transposition


## Convert alignment graph to numpy masked array

In [13]:
vector_space_as_ma = convert_alignment_graph_to_numpy_masked_array(hyperedges, witness_sigla)

## Create variant graph from alignment hypergraph

First create variant graph nodes. Walk over witness nodes in `witness_node_lists`. Keep track of which hyperedges have been added to variant graph. For each witness node:

1. If node has no hyperedge, create variant graph node with selected information from witness node on it.
1. If node has new hyperedge, create variant graph node with selected information from all witness nodes from hyperedge on it.
1. If node has hyperedge that is already on variant graph, do nothing.

Witness nodes contain token string, a witness siglum, a witness offset, and an optional hyperedge. The hyperedge contains, as dictionary items, all witness sigla as keys with witness nodes as values. From this information, we put onto the variant graph node the token string plus the hyperedge dictionary with sigla as keys and witness offsets as values.

**Notes:**

* The variant graph does not need to contain the token string, since we can look it up later, but for ease of visualization we include it here.
* In Real Life CollateX a variant graph node contains properties other than the token string. We ignore that structure in our experiment as we concentrate on visualization.

In [14]:
class VG_node():
    """Variant graph node"""
    def __init__(self, token_string, data): # dictionary of siglum:witness_offset
        self.token_string = token_string
        self.__dict__.update(**data)
        self._sigla = [key for key in data.keys()]
    def __repr__(self):
        return self.token_string + "~" + "|".join([":".join([str(key), str(getattr(self, key))]) for key in self.sigla()])
    def __setitem__(self, key, value):
        self._sigla.append(key)
        self.__dict__[key] = value
    def __getitem__(self, key):
        return self.__dict__[key]
    def __contains__(self, key):
        return key in self.__dict__
    def sigla(self):
        return self._sigla
    def values(self):
        return [self[key] for key in self._sigla]
    def items(self):
        return [(key, self.__dict__[key]) for key in self._sigla]

In [15]:
# Create networkx variant graph from our (not networkx) alignment graph
# Create variant graph edges for start and end nodes (only)
# We've treated VG as a global and we refer to it in the earlier functions; should we pass it instead?
def augment_or_add_edge_without_conversion(siglum, source_VG_node, target_VG_node):
    if VG.has_edge(source_VG_node, target_VG_node):
        VG[source_VG_node][target_VG_node]["siglum"].append(siglum)
    else:
        VG.add_edge(
            source_VG_node,
            target_VG_node,
            siglum = [siglum]
        )

In [16]:
# Use alignment graph node to look up variant graph node
def alignment_node_to_VG_node(alignment_node: WitnessNode) -> VG_node:
    global witness_offset_to_VG_node
    return witness_offset_to_VG_node[alignment_node.witness][alignment_node.witness_offset]

In [17]:
# Create variant graph data edges (except start and end nodes)
def augment_or_add_edge(siglum, source, target):
    source_VG_node = alignment_node_to_VG_node(source)
    target_VG_node = alignment_node_to_VG_node(target)
    if VG.has_edge(source_VG_node, target_VG_node):
        VG[source_VG_node][target_VG_node]["siglum"].append(siglum)
    else:
        VG.add_edge(
            source_VG_node,
            target_VG_node,
            siglum = [siglum]
                )

In [18]:
def create_variant_graph_from_vector_space():
    # create variant graph and add start and end nodes

    start_node = VG_node("START", {})
    end_node = VG_node("END", {})
    global VG # does this have to be global; we return it at the end, but we also use it in functions above
    VG = nx.DiGraph(start = start_node, end = end_node) # create start and end properties to find terminal nodes
    VG.add_node(start_node)
    VG.add_node(end_node)

    # keep track of which witness nodes (and therefore also hyperedges)
    # have been added to VG
    # does this duplicate information available from the following structure, since
    #   if we assign a value below to replace the None, this doesn't seem to add anything
    #   Can we get rid of this and use witness_offset_to_VG_node for this purpose instead?
    # NB: Includes START and END, which we will later ignore
    from bitarray import bitarray
    global VG_tracking # keep this, but should we return it instead of making it global?
    VG_tracking = {}
    for siglum, witness_node_list in witness_node_lists.items():
        VG_tracking[siglum] = bitarray(len(witness_node_list))
        VG_tracking[siglum].setall(0)

    # map from witness token node to variant graph node (needed to construct edges)
    # values will be added as we create variant graph nodes
    # NB: Includes START and END, which we will later ignore
    global witness_offset_to_VG_node # does this have to be global?
    witness_offset_to_VG_node = {}
    for siglum in witness_node_lists.keys():
        witness_offset_to_VG_node[siglum] = [None] * (len(witness_node_lists[siglum]))

    # Replaces old code in following cell to build variant graph,
    #   this time using vector space as main data source

    # 1. Convert vectors to variant graph nodes

    for row in vector_space_as_ma:
        print(' '.join(('Processing', str(row))))
        # create dictionary of siglum:value for node (variable name: data) and update globals
        data = {}
        for index, value in enumerate(row):
            if value != vector_space_as_ma.fill_value:
                siglum = 'w' + str(index)
                value = int(value)
                data[siglum] = value # add to eventually data for new VG node
                VG_tracking[siglum][value] = 1 # update global; do we need to adjust by 1?
        # get token string for node (variable name: token_string)
        siglum, offset = next(iter(data.items()))
        token_string = witness_node_lists[siglum][int(offset) + 1].token_string # CHECK THIS: Do we need to add 1?
        # create and add new VG_node
        new_VG_node = VG_node(token_string, data)
        VG.add_node(new_VG_node)
        for siglum, offset in data.items(): # Eek! Another for loop! How embarrassing!
            witness_offset_to_VG_node[siglum][offset] = new_VG_node # update other global; CHECK THIS: do we need to adjust by 1?

    # 2. Traverse variant graph (in arbitrary order), each of which contains witness nodes.
    #    Draw outgoing edges, which point to nodes with next tokens in each witness present
    #      on the variant graph node.
    #    Creating new variant graph nodes for witness tokens not in a hyperedge.
    #    Use queue because we're iterating over a dynamic structure (inventory of variant graph nodes).

    VG_node_queue = queue.Queue()
    #
    # Temporarily skipping START and END, which are the first two nodes
    #
    for node in VG.nodes(): # add all initial VG nodes to queue except START and END
        VG_node_queue.put(node)
        print(f"Adding node to VG_node_queue; length is {VG_node_queue.qsize()=}")
    ignore_start = VG_node_queue.get() # temporarily ignore START node when creating edges
    ignore_end = VG_node_queue.get() # temporarily ignore END node when creating edges
    print(f"{ignore_start=} {ignore_end=}")
    print(f"Removed START and END; length is {VG_node_queue.qsize()=}")
    while not VG_node_queue.empty():
        print(f"Processing source node from VG_node_queue; length is {VG_node_queue.qsize()=}")
        source_node = VG_node_queue.get()
        targets = set()
        edge_labels = defaultdict(list) # key is target VG node, value is list of sigla
        sigla = source_node._sigla # all sigla on source node
        for siglum in sigla:
            # target may or may not already exist as node in VG
            source_offset = source_node[siglum]
            target_offset = source_offset + 1
            target = witness_offset_to_VG_node[siglum][target_offset]
            if target: # does the target already exist?:
                targets.add(target)
                edge_labels[target].append(siglum)
            else: # 1) create target, 2) add to targets, 3) add to queue of nodes, and 4 + 5) update globals
                # TODO: Do we need to update globals? We shouldn't need to return to nodes we add here.
                # NB: If real target object is of type WitnessNodeEnd, don't add it to the queue or create an edge
                # print(f"{type(witness_node_lists[siglum][target_offset])=}")
                ## TODO: PAY ATTENTION TO THE + 1 AT THE END! WHY IS IT NEEDED?
                witness_node_target = witness_node_lists[siglum][target_offset + 1]
                if type(witness_node_target) != WitnessNodeEnd:
                    target_token_string = witness_node_target.token_string
                    new_VG_node = VG_node(target_token_string, {siglum: target_offset}) # 1 create target
                    targets.add(new_VG_node) # 2 add to targets
                    edge_labels[new_VG_node].append(siglum)
                    VG_node_queue.put(new_VG_node) # 3 add to queue of nodes
                    VG_tracking[siglum][target_offset] = 1 # 4 update first global
                    witness_offset_to_VG_node[siglum][target_offset] = new_VG_node # 5. update second global
        for target in targets:
            VG.add_edge(source_node, target, label=",".join(edge_labels[target]))

    # we're done! return the result
    return VG

In [19]:
# # recover some memory
# del witness_node_list
# del witness_offset_to_VG_node
# del VG_tracking
gc.collect()
VG = create_variant_graph_from_vector_space()

0

Processing [53.0 52.0 52.0 55.0 55.0 55.0]
Processing [54.0 53.0 53.0 56.0 56.0 56.0]
Processing [55.0 54.0 54.0 57.0 57.0 57.0]
Processing [56.0 55.0 55.0 58.0 58.0 58.0]
Processing [57.0 56.0 56.0 59.0 59.0 59.0]
Processing [58.0 57.0 57.0 60.0 60.0 60.0]
Processing [59.0 58.0 58.0 61.0 61.0 61.0]
Processing [60.0 59.0 59.0 62.0 62.0 62.0]
Processing [61.0 60.0 60.0 63.0 63.0 63.0]
Processing [62.0 61.0 61.0 64.0 64.0 64.0]
Processing [63.0 62.0 62.0 65.0 65.0 65.0]
Processing [64.0 63.0 63.0 66.0 66.0 66.0]
Processing [65.0 64.0 64.0 67.0 67.0 67.0]
Processing [66.0 65.0 65.0 68.0 68.0 68.0]
Processing [67.0 66.0 66.0 69.0 69.0 69.0]
Processing [68.0 67.0 67.0 70.0 70.0 70.0]
Processing [69.0 68.0 68.0 71.0 71.0 71.0]
Processing [70.0 69.0 69.0 72.0 72.0 72.0]
Processing [71.0 70.0 70.0 73.0 73.0 73.0]
Processing [72.0 71.0 71.0 74.0 74.0 74.0]
Processing [73.0 72.0 72.0 75.0 75.0 75.0]
Processing [74.0 73.0 73.0 76.0 76.0 76.0]
Processing [75.0 74.0 74.0 77.0 77.0 77.0]
Processing 


Processing source node from VG_node_queue; length is VG_node_queue.qsize()=233
Processing source node from VG_node_queue; length is VG_node_queue.qsize()=232
Processing source node from VG_node_queue; length is VG_node_queue.qsize()=231
Processing source node from VG_node_queue; length is VG_node_queue.qsize()=230
Processing source node from VG_node_queue; length is VG_node_queue.qsize()=229
Processing source node from VG_node_queue; length is VG_node_queue.qsize()=228
Processing source node from VG_node_queue; length is VG_node_queue.qsize()=227
Processing source node from VG_node_queue; length is VG_node_queue.qsize()=226
Processing source node from VG_node_queue; length is VG_node_queue.qsize()=225
Processing source node from VG_node_queue; length is VG_node_queue.qsize()=224
Processing source node from VG_node_queue; length is VG_node_queue.qsize()=223
Processing source node from VG_node_queue; length is VG_node_queue.qsize()=222
Processing source node from VG_node_queue; length i

## Visualize variant graph

In [20]:
# # node id values must be strings for graphviz
a = graphviz.Digraph(format="svg", name="variant_graph_unjoined")
a.attr(rankdir = "LR")
a.attr(rank = 'same')
a.attr(compound='true')

# plot nodes, building {node: id} for lookup
node_to_id = {}
for index, node in enumerate(VG.nodes()):
    node_id = str(index)
    node_to_id[node] = node_id
    node_text = node.token_string + " (" + node_id + ")"
    a.node(node_id, label=node_text)

# plot edges
for edge in VG.edges(data=True):
    # edge is a three-item tuple: source, target, dictionary of properties
    a.edge(node_to_id[edge[0]], node_to_id[edge[1]], label=edge[2]["label"])

# print('aligning', how_many_paragraphs, 'paragraphs') # confirm
SVG(a.view())

In [21]:
# #  This function joins the variant graph in place.
# #  This function is a straight port of the Java version of CollateX.

# def join(graph):
#     processed = set()
#     end = graph.graph["end"]
#     queue = deque()
#     for (_, neighbor) in graph.out_edges(graph.graph["start"]):
#         queue.appendleft(neighbor)
#     while queue:
#         vertex = queue.popleft()
#         out_edges = graph.out_edges(vertex)
#         if len(out_edges) == 1:
#             (_, join_candidate) = next(iter(out_edges))
#             can_join = join_candidate != end and len(graph.in_edges(join_candidate)) == 1
#             if can_join:
#                 join_vertex_and_join_candidate(graph, join_candidate, vertex)
#                 for (_, neighbor, data) in list(graph.out_edges(join_candidate, data=True)):
#                     graph.remove_edge(join_candidate, neighbor)
#                     if graph.has_edge(vertex, neighbor):
#                         graph[source][target]["siglum"].append(data["siglum"])
#                     else:
#                         graph.add_edge(vertex, neighbor, siglum=data["siglum"])
#                 graph.remove_edge(vertex, join_candidate)
#                 graph.remove_node(join_candidate)
#                 queue.appendleft(vertex)
#                 continue
#         processed.add(vertex)
#         for (_, neighbor) in out_edges:
#             # FIXME: Why do we run out of memory in some cases here, if this is not checked?
#             if neighbor not in processed:
#                 queue.appendleft(neighbor)


# def join_vertex_and_join_candidate(graph, join_candidate, vertex):
#     # Note: since there is no normalized/non normalized content in the graph
#     # a space character is added here for non punctuation tokens

#     if re.match(r'^\W', join_candidate.token_string):
#         vertex.token_string += join_candidate.token_string
#     else:
#         vertex.token_string += (" " + join_candidate.token_string)
#     # join_candidate must have exactly one token (inside a list); left item may have more
# #     for siglum, token in join_candidate.tokens.items():
# #         vertex.add_token(siglum, token[0])

In [22]:
# len(nx.algorithms.cycles.find_cycle(VG))

In [23]:
# nx.algorithms.cycles.find_cycle(VG)

In [24]:
# join(VG)

In [25]:
# pp.pprint([edge for edge in VG.edges()])

In [26]:
# # node id values must be strings for graphviz
# a = graphviz.Digraph(format="svg", name="variant_graph_joined")
# a.attr(rankdir = "LR")
# a.attr(rank = 'same')
# a.attr(compound='true')

# # plot nodes, building {node: id} for lookup
# node_to_id = {}
# for index, node in enumerate(VG.nodes()):
#     node_id = str(index)
#     node_to_id[node] = node_id
#     node_text = node.token_string + " (" + node_id + ")"
#     a.node(node_id, label=node_text)

# # plot edges
# for edge in VG.edges(data=True):
#     # edge is a three-item tuple: source, target, dictionary of properties
#     label = "(all)" if len(edge[2]["siglum"]) == len(witness_sigla) else ",".join(sorted(edge[2]["siglum"]))
#     a.edge(node_to_id[edge[0]], node_to_id[edge[1]], label=label)

# print('aligning', how_many_paragraphs, 'paragraphs') # confirm
# SVG(a.view())

In [27]:
# nx.algorithms.cycles.find_cycle(VG)

# To do next

1. Move SVG/Graphviz code into function
1. Create alignment table visualization without joining
1. Add joining to the alignment table (to check accuracy of joining results)
1. Edge labels are wrong, which may be a problem with the join() function
1. Test intermediate data sets, larger than a paragraph and smaller than a chapter
1. Reassess method of prioritizing blocks
1. Implement decision tree / graph

In [28]:
hyperedges

[w0:START|w1:START|w2:START|w3:START|w4:START|w5:START,
 w0:END|w1:END|w2:END|w3:END|w4:END|w5:END,
 w0:we|w1:we|w2:we|w3:we|w4:we|w5:we,
 w0:reflect|w1:reflect|w2:reflect|w3:reflect|w4:reflect|w5:reflect,
 w0:on|w1:on|w2:on|w3:on|w4:on|w5:on,
 w0:the|w1:the|w2:the|w3:the|w4:the|w5:the,
 w0:vast|w1:vast|w2:vast|w3:vast|w4:vast|w5:vast,
 w0:diversity|w1:diversity|w2:diversity|w3:diversity|w4:diversity|w5:diversity,
 w0:of|w1:of|w2:of|w3:of|w4:of|w5:of,
 w0:the|w1:the|w2:the|w3:the|w4:the|w5:the,
 w0:plants|w1:plants|w2:plants|w3:plants|w4:plants|w5:plants,
 w0:and|w1:and|w2:and|w3:and|w4:and|w5:and,
 w0:animals|w1:animals|w2:animals|w3:animals|w4:animals|w5:animals,
 w0:which|w1:which|w2:which|w3:which|w4:which|w5:which,
 w0:have|w1:have|w2:have|w3:have|w4:have|w5:have,
 w0:been|w1:been|w2:been|w3:been|w4:been|w5:been,
 w0:cultivated,|w1:cultivated,|w2:cultivated,|w3:cultivated,|w4:cultivated,|w5:cultivated,,
 w0:and|w1:and|w2:and|w3:and|w4:and|w5:and,
 w0:which|w1:which|w2:which|w3:whi

In [29]:
len(hyperedges)

247

In [30]:
vector_space_as_ma.shape

(245, 6)

In [31]:
vector_space_as_ma

masked_array(
  data=[[53.0, 52.0, 52.0, 55.0, 55.0, 55.0],
        [54.0, 53.0, 53.0, 56.0, 56.0, 56.0],
        [55.0, 54.0, 54.0, 57.0, 57.0, 57.0],
        [56.0, 55.0, 55.0, 58.0, 58.0, 58.0],
        [57.0, 56.0, 56.0, 59.0, 59.0, 59.0],
        [58.0, 57.0, 57.0, 60.0, 60.0, 60.0],
        [59.0, 58.0, 58.0, 61.0, 61.0, 61.0],
        [60.0, 59.0, 59.0, 62.0, 62.0, 62.0],
        [61.0, 60.0, 60.0, 63.0, 63.0, 63.0],
        [62.0, 61.0, 61.0, 64.0, 64.0, 64.0],
        [63.0, 62.0, 62.0, 65.0, 65.0, 65.0],
        [64.0, 63.0, 63.0, 66.0, 66.0, 66.0],
        [65.0, 64.0, 64.0, 67.0, 67.0, 67.0],
        [66.0, 65.0, 65.0, 68.0, 68.0, 68.0],
        [67.0, 66.0, 66.0, 69.0, 69.0, 69.0],
        [68.0, 67.0, 67.0, 70.0, 70.0, 70.0],
        [69.0, 68.0, 68.0, 71.0, 71.0, 71.0],
        [70.0, 69.0, 69.0, 72.0, 72.0, 72.0],
        [71.0, 70.0, 70.0, 73.0, 73.0, 73.0],
        [72.0, 71.0, 71.0, 74.0, 74.0, 74.0],
        [73.0, 72.0, 72.0, 75.0, 75.0, 75.0],
        [74.0, 73.0,

In [32]:
for node in VG.nodes:
    pp.pprint(node)

START~
END~
we~w0:53|w1:52|w2:52|w3:55|w4:55|w5:55
reflect~w0:54|w1:53|w2:53|w3:56|w4:56|w5:56
on~w0:55|w1:54|w2:54|w3:57|w4:57|w5:57
the~w0:56|w1:55|w2:55|w3:58|w4:58|w5:58
vast~w0:57|w1:56|w2:56|w3:59|w4:59|w5:59
diversity~w0:58|w1:57|w2:57|w3:60|w4:60|w5:60
of~w0:59|w1:58|w2:58|w3:61|w4:61|w5:61
the~w0:60|w1:59|w2:59|w3:62|w4:62|w5:62
plants~w0:61|w1:60|w2:60|w3:63|w4:63|w5:63
and~w0:62|w1:61|w2:61|w3:64|w4:64|w5:64
animals~w0:63|w1:62|w2:62|w3:65|w4:65|w5:65
which~w0:64|w1:63|w2:63|w3:66|w4:66|w5:66
have~w0:65|w1:64|w2:64|w3:67|w4:67|w5:67
been~w0:66|w1:65|w2:65|w3:68|w4:68|w5:68
cultivated,~w0:67|w1:66|w2:66|w3:69|w4:69|w5:69
and~w0:68|w1:67|w2:67|w3:70|w4:70|w5:70
which~w0:69|w1:68|w2:68|w3:71|w4:71|w5:71
have~w0:70|w1:69|w2:69|w3:72|w4:72|w5:72
varied~w0:71|w1:70|w2:70|w3:73|w4:73|w5:73
during~w0:72|w1:71|w2:71|w3:74|w4:74|w5:74
all~w0:73|w1:72|w2:72|w3:75|w4:75|w5:75
ages~w0:74|w1:73|w2:73|w3:76|w4:76|w5:76
under~w0:75|w1:74|w2:74|w3:77|w4:77|w5:77
the~w0:76|w1:75|w2:75|w3:78|w

In [33]:
pp.pprint(witness_offset_to_VG_node)

{ 'w0': [ WHEN~w0:0|w1:0|w2:0|w3:3|w4:3|w5:3,
          we~w0:1|w1:1|w2:1|w3:4|w4:4|w5:4,
          look~w0:2|w1:2|w2:2|w3:5,
          to~w0:3|w1:3|w2:3|w3:6,
          the~w0:4|w1:4|w2:4|w3:7|w4:6|w5:6,
          individuals~w0:5|w1:5|w2:5|w3:8|w4:7|w5:7,
          of~w0:6|w1:6|w2:6|w3:9|w4:8|w5:8,
          the~w0:7|w1:7|w2:7|w3:10|w4:9|w5:9,
          same~w0:8|w1:8|w2:8|w3:11|w4:10|w5:10,
          variety~w0:9|w1:9|w2:9|w3:12|w4:11|w5:11,
          or~w0:10|w1:10|w2:10|w3:13|w4:12|w5:12,
          sub-variety~w0:11|w1:11|w2:11|w3:14|w4:13|w5:13,
          of~w0:12|w1:12|w2:12|w3:15|w4:14|w5:14,
          our~w0:13|w1:13|w2:13|w3:16|w4:15|w5:15,
          older~w0:14|w1:14|w2:14|w3:17|w4:16|w5:16,
          cultivated~w0:15|w1:15|w2:15|w3:18|w4:17|w5:17,
          plants~w0:16|w1:16|w2:16|w3:19|w4:18|w5:18,
          and~w0:17|w1:17|w2:17|w3:20|w4:19|w5:19,
          animals,~w0:18|w1:18|w2:18|w3:21|w4:20|w5:20,
          one~w0:19|w1:19|w2:19|w3:22|w4:21|w5:21,
          of~w0:20

,
          of~w0:174|w1:173|w2:173|w3:176|w4:169|w5:167,
          variation;~w0:175|w1:174|w2:174|w3:177|w4:170|w5:168,
          and~w0:176|w1:175|w2:175|w3:178|w4:171|w5:169,
          that,~w3:179|w4:172|w5:170,
          when~w0:178|w1:177|w2:177|w3:180|w4:173|w5:171,
          the~w0:179|w1:178|w2:178|w3:181|w4:174|w5:172,
          organisation~w0:180|w1:179|w2:179|w3:182|w4:175|w5:173,
          has~w0:181|w1:180|w2:180|w3:183|w4:176|w5:174,
          once~w0:182|w1:181|w2:181|w3:184|w4:177|w5:175,
          begun~w0:183|w1:182|w2:182|w3:185|w4:178|w5:176,
          

to~w0:184|w1:183|w2:183|w3:186|w4:179|w5:177,
          vary,~w0:185|w1:184|w2:184|w3:187|w4:180|w5:178,
          it~w0:186|w1:185|w2:185|w3:188|w4:181|w5:179,
          generally~w0:187|w1:186|w2:186|w3:189|w4:182|w5:180,
          continues~w0:188|w1:187|w2:187|w3:190|w4:183|w5:181,
          to~w0:189|w1:188|w2:188|w3:191,
          vary~w0:190|w1:189|w2:189|w3:192,
          for~w0:191|w1:190|w2:190|w3:193|w4:185|w5:183,
          many~w0:192|w1:191|w2:191|w3:194|w4:186|w5:184,
          generations.~w0:193|w1:192|w2:192|w3:195|w4:187|w5:185,
          No~w0:194|w1:193|w2:193|w3:196|w4:188|w5:186,
          case~w0:195|w1:194|w2:194|w3:197|w4:189|w5:187,
          is~w0:196|w1:195|w2:195|w3:198|w4:190|w5:188,
          on~w0:197|w1:196|w2:196|w3:199|w4:191|w5:189,
          record~w0:198|w1:197|w2:197|w3:200|w4:192|w5:190,
          of~w0:199|w1:198|w2:198|w3:201|w4:193|w5:191,
          a~w0:200|w1:199|w2:199|w3:202|w4:194|w5:192,
          variable~w0:201|w1:200|w2:200|w3:203|w4

new~w0:219|w1:218|w2:218|w3:221|w4:211|w5:209,
          varieties:~w0:220|w1:219|w2:219|w3:222|w4:212|w5:210,
          our~w0:221|w1:220|w2:220|w3:223|w4:213|w5:211,
          oldest~w0:222|w1:221|w2:221|w3:224|w4:214|w5:212,
          domesticated~w0:223|w1:222|w2:222|w3:225|w4:215|w5:213,
          animals~w0:224|w1:223|w2:223|w3:226|w4:216|w5:214,
          are~w0:225|w1:224|w2:224|w3:227|w4:217|w5:215,
          still~w0:226|w1:225|w2:225|w3:228|w4:218|w5:216,
          capable~w0:227|w1:226|w2:226|w3:229|w4:219|w5:217,
          of~w0:228|w1:227|w2:227|w3:230|w4:220|w5:218,
          rapid~w0:229|w1:228|w2:228|w3:231|w4:221|w5:219,
          improvement~w0:230|w1:229|w2:229|w3:232|w4:222|w5:220,
          or~w0:231|w1:230|w2:230|w3:233|w4:223|w5:221,
          modification.~w0:232|w1:231|w2:231|w3:234|w4:224|w5:222,
          None,
          None]}
